## 1) Introduction

This simple notebook implements the mathematical forward model whose description can be found [in this document](https://www.overleaf.com/project/623dc6078b3a6b1b29dfcc5a) (needs to be invited).

In [77]:
# The modules to import

from os import path
import numpy as np
from netCDF4 import Dataset

## 2) Input setup from netCDF4 file

Concerning the dimensions, like in the document we have :
* `N_i` the amount of pixels in a column;
* `N_j` the amount of pixels in a row;
* `N_ij` the product of `N_i` and `N_j`;
* `N_k` the amount of channels.

We retreive the input scene from a netCDF4 file and put it in the 3 dimensional matrix `Ux`, corresponding to the $U^{[x]}$ matrix in the document.

Then we instanciate the matrix `X`, which is `Ux` whose 2nd space dimension is flattened (corresponding to $X$).

Finally we get the `spectral_stencil`, the ordered array of the sampled wavelengths (in Ångström).

In [78]:
netCDF4_path = path.join("data", "multispectral_colorchecker.ns")

dataset = Dataset(netCDF4_path, 'r')
dataset.set_auto_mask(False)

# Declaration of Ux
Ux = dataset.groups['radiance group'].variables['radiance matrix'][:, :, :].astype(float)
Ux = np.arange(3600).reshape(10, 6, 60)

# Declaration of the spectral stencil
spectral_stencil = dataset.variables['spectral stencil'][:]

# Declaration of the dimensions
N_i, N_j, N_k = Ux.shape
N_ij = N_i * N_j

# Declaration of X
X = np.zeros((N_ij, N_k))

for k in range(N_k):
    X[:, k] = Ux[:, :, k].flatten('F')

## 3) Mask creation

In this part we declare `k_r`, `k_g` and `k_b`, the indices of the colors red, green and blue in the spectral stencil.

Then we construct the 3 dimensional matrices `UBayer` and `UQuad`. They are the implementation of the Bayer CFA and the Quad-Bayer CFA.

See $U^{[Bayer]}$ and $U^{[Quad]}$ in the document for more informations.

Like for `X`, we introduce `HBayer` and `HQuad`, the flatened versions of `UBayer` and `UQuad`.

We compute the two mask matrices with thoe expressions :

$$

\mathcal{U}^{[Bayer]}_{i, j, k} =
\left\{
\begin{array}{lll}
    1 & \text{if $k = k_r$ and} & \text{$i \bmod{2} = 0$ and $j \bmod{2} = 1$}\\
    1 & \text{if $k = k_g$ and} & \text{(($i \bmod{2} = 0$ and $j \bmod{2} = 0$) or}\\
    && \text{($i \bmod{2} = 1$ and $j \bmod{2} = 1$))}\\
    1 & \text{if $k = k_b$ and} & \text{$i \bmod{2} = 1$ and $j \bmod{2} = 0$}\\
    0 & \text{otherwise}
\end{array}
\right.

\text{and} \quad

\mathcal{U}^{[Quad]}_{i, j, k} =
\left\{
\begin{array}{lll}
    1 & \text{if $k = k_r$ and} & \text{$i \bmod{4} < 2$ and $j \bmod{4} \geq 2$}\\
    1 & \text{if $k = k_g$ and} & \text{(($i \bmod{4} < 2$ and $j \bmod{4} < 2$) or}\\
    && \text{($i \bmod{4} \geq 2$ and $j \bmod{4} \geq 2$))}\\
    1 & \text{if $k = k_b$ and} & \text{$i \bmod{4} \geq 2$ and $j \bmod{4} < 2$}\\
    0 & \text{otherwise}
\end{array}
\right.

$$

In [79]:
# Declaration of the indices of the wavelength of the red, green and blue colors
k_r = (np.abs(spectral_stencil - 6500)).argmin()
k_g = (np.abs(spectral_stencil - 5500)).argmin()
k_b = (np.abs(spectral_stencil - 4450)).argmin()

# Declaration of UBayer and UQuad
UBayer = np.zeros_like(Ux)
UQuad = np.zeros_like(Ux)

for i in range(N_i):
    for j in range(N_j):
        for k in range(N_k):
            if k == k_r:
                if i % 2 == 0 and j % 2 == 1:
                    UBayer[i, j, k] = 1

                if i % 4 < 2 and j % 4 >= 2:
                    UQuad[i, j, k] = 1

            elif k == k_g:
                if (i % 2 == 0 and j % 2 == 0) or (i % 2 == 1 and j % 2 == 1):
                    UBayer[i, j, k] = 1

                if (i % 4 < 2 and j % 4 < 2) or (i % 4 >= 2 and j % 4 >= 2):
                    UQuad[i, j, k] = 1

            elif k == k_b:
                if i % 2 == 1 and j % 2 == 0:
                    UBayer[i, j, k] = 1

                if i % 4 >= 2 and j % 4 < 2:
                    UQuad[i, j, k] = 1

# Declaration of the flattened mask matrices
HBayer = np.zeros((N_ij, N_k))
HQuad = np.zeros((N_ij, N_k))

for k in range(N_k):
    HBayer[:, k] = UBayer[:, :, k].flatten('F')
    HQuad[:, k] = UQuad[:, :, k].flatten('F')

## 4) Application of the masks

We use the formula :

$$
    y = \sum_{k = 1}^{N_k} X_{:k} \odot H_{:k}
$$

Where $y$ is a vector of $\mathbb{R}^{N_{ij}}$.

In [86]:
# Results of the masks
yBayer = sum(X[:, k] * HBayer[:, k] for k in range(N_k))
yQuad = sum(X[:, k] * HQuad[:, k] for k in range(N_k))

# Un-flatten the results
YBayer = np.reshape(yBayer, (N_i, N_j), order='F')
YQuad = np.reshape(yQuad, (N_i, N_j), order='F')

## 5) To summarize

In [87]:
print(f"The spectral stencil goes from {int(spectral_stencil[0] / 10)}nm to {int(spectral_stencil[-1] / 10)}nm with a step of {int((spectral_stencil[1] - spectral_stencil[0]) / 10)}nm. Which corresponds to {N_k} sampled values.")

print(f"The \'expanded\' marices Ux, UBayer and UQuad are of size : {N_i}x{N_j}x{N_k}.")

print(f"The flattened matrices X, HBayer and HQuad are of size : {N_ij}x{N_k}.")

print(f"The flattened results yBayer and yQuad are of size : {N_ij}.")

print(f"The \'expanded\' matrices YBayer and YQuad, which represents well the values of the pixels on the sensor at the end, are of size : {N_i}x{N_j}.")

The spectral stencil goes from 400nm to 990nm with a step of 10nm. Which corresponds to 60 sampled values.
The 'expanded' marices Ux, UBayer and UQuad are of size : 10x6x60.
The flattened matrices X, HBayer and HQuad are of size : 60x60.
The flattened results yBayer and yQuad are of size : 60.
The 'expanded' matrices YBayer and YQuad, which represents well the values of the pixels on the sensor at the end, are of size : 10x6.
